In [84]:
import pandas as pd
import numpy as np

# 导入数据
x_all = pd.read_csv('data/random_data_regression_X.csv', header=None)
y_all = pd.read_csv('data/random_data_regression_y.csv', header=None)
train_x = x_all[:40]
train_y = y_all[:40]
test_x = x_all[40:]
test_y = y_all[40:]
(x_all.shape, y_all.shape)

((50, 500), (50, 1))

In [85]:
import time
from sklearn.metrics import mean_squared_error


# 封装梯度下降法(包含正则化项)
class GradientDescentLinerRegressionWithRegularization:
    def __init__(self, learning_rate=0.01, max_iter=1000, _lambda=0.1, early_stop=True, seed=None):
        np.random.seed(seed)
        self.early_stop = early_stop
        self._lambda = _lambda
        self.lr = learning_rate  # 设置学习率
        self.max_iter = max_iter
        self.loss_arr = []
        self.intercept_ = None
        self.coef_ = None

    def fit(self, x, y):
        self.x = x
        self.x_b = np.hstack([np.ones((len(x), 1)), x])
        self.x_b_dim = np.size(self.x_b, 1)
        self.x_sample = np.size(self.x_b, 0)
        self.y = y
        self.w = np.random.normal(1, 0.001, (self.x_b_dim, 1))
        for i in range(self.max_iter):
            self._train_step()
            self.loss_arr.append(self.loss())
            print("iter: {}, loss_in_train: {:.5f}, loss_in_test: {:.5f}".format(i + 1, self.loss(is_test=False), self.loss_arr[-1]))

            # 连续十次不下降则退出
            if self.early_stop and len(self.loss_arr) > 10:
                arr = np.array(self.loss_arr[:-10])
                if np.argmin(arr) == 0:
                    return

    def _f(self, x, w):
        return x.dot(w)

    def predict(self, x=None):
        if x is None:
            x = self.x_b
        x = np.hstack([np.ones((len(x), 1)), x])
        y_pred = self._f(x, self.w)
        return y_pred

    def loss(self, is_test=True, y_true=None, y_pred=None):
        if y_true is None or y_pred is None:
            y_true = self.y
        if is_test:
            return np.sqrt(mean_squared_error(test_y, self.predict(test_x)))
        else:
            return np.sqrt(mean_squared_error(y_true, self.predict(self.x)))

    def _calc_gradient(self):
        d_w = np.empty(self.x_b_dim).reshape(-1, 1)
        d_w[0] = np.sum(self.x_b.dot(self.w) - self.y)
        for i in range(1, self.x_b_dim):
            # print("{} {} {} {}".format(self.x_b.shape, self.w.shape, self.y.shape, self.x_b[:, i].T.shape))
            d_w[i] = np.squeeze((self.x_b.dot(self.w) - self.y)).dot(self.x_b[:, i].T)
        return d_w * 2 / self.x_sample + (self._lambda / self.x_sample * self.w)

    def _train_step(self):
        d_w = self._calc_gradient()
        self.w = self.w - self.lr * d_w
        self.intercept_ = self.w[0]
        self.coef_ = self.w[1:]
        return self.w

In [86]:
time_start = time.time()  # 记录开始时间
reg = GradientDescentLinerRegressionWithRegularization(learning_rate=1e-5, max_iter=30, _lambda=100, early_stop=True, seed=1024)
reg.fit(train_x, train_y)
print("运行时间: ", time.time() - time_start)

iter: 1, loss_in_train: 30.80939, loss_in_test: 35.08037
iter: 2, loss_in_train: 30.80125, loss_in_test: 35.08077
iter: 3, loss_in_train: 30.79312, loss_in_test: 35.08118
iter: 4, loss_in_train: 30.78499, loss_in_test: 35.08158
iter: 5, loss_in_train: 30.77686, loss_in_test: 35.08198
iter: 6, loss_in_train: 30.76874, loss_in_test: 35.08238
iter: 7, loss_in_train: 30.76061, loss_in_test: 35.08278
iter: 8, loss_in_train: 30.75249, loss_in_test: 35.08319
iter: 9, loss_in_train: 30.74437, loss_in_test: 35.08359
iter: 10, loss_in_train: 30.73626, loss_in_test: 35.08399
iter: 11, loss_in_train: 30.72815, loss_in_test: 35.08439
运行时间:  1.5480129718780518


In [87]:
time_start = time.time()  # 记录开始时间
reg = GradientDescentLinerRegressionWithRegularization(learning_rate=1e-5, max_iter=30, _lambda=100, early_stop=False, seed=1024)
rate = np.diag([1] * 500)
rate[0][0] = 100000
train_x = train_x.dot(rate)
test_x = test_x.dot(rate)
reg.fit(train_x, train_y)
print("运行时间: ", time.time() - time_start)

iter: 1, loss_in_train: 15989449595.48327, loss_in_test: 19107178862.43552
iter: 2, loss_in_train: 2754287787120579.50000, loss_in_test: 3291337144231060.50000
iter: 3, loss_in_train: 474444174515207274496.00000, loss_in_test: 566954456156685139968.00000
iter: 4, loss_in_train: 81726127452621944486100992.00000, loss_in_test: 97661631510259090965135360.00000
iter: 5, loss_in_train: 14077862617297495002779842772992.00000, loss_in_test: 16822857930954745121414521552896.00000
iter: 6, loss_in_train: 2425004365787357994633132608973701120.00000, loss_in_test: 2897847850671608891980680261953126400.00000
iter: 7, loss_in_train: 417722940900288747644712018932248529076224.00000, loss_in_test: 499173339043081383236297770412926434279424.00000
iter: 8, loss_in_train: 71955522149227679295953406822923933966047117312.00000, loss_in_test: 85985888580613404241229189804493876341613002752.00000
iter: 9, loss_in_train: 12394811634259514926228986790270247729448618234478592.00000, loss_in_test: 1481163446984

/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
/Users/owemshu/opt/anaconda3/envs/deeplearning/lib/python3.9/site-packages/sklearn/metrics/_regression.py:446: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)


In [88]:
from sklearn.preprocessing import Normalizer

time_start = time.time()  # 记录开始时间
reg = GradientDescentLinerRegressionWithRegularization(learning_rate=1e-5, max_iter=30, _lambda=100, early_stop=False, seed=1024)

nor = Normalizer()
train_x = nor.fit_transform(train_x)
test_x = nor.transform(test_x)

reg.fit(train_x, train_y)
print("运行时间: ", time.time() - time_start)

iter: 1, loss_in_train: 22.27634, loss_in_test: 30.74151
iter: 2, loss_in_train: 22.27634, loss_in_test: 30.74151
iter: 3, loss_in_train: 22.27634, loss_in_test: 30.74151
iter: 4, loss_in_train: 22.27634, loss_in_test: 30.74151
iter: 5, loss_in_train: 22.27634, loss_in_test: 30.74151
iter: 6, loss_in_train: 22.27635, loss_in_test: 30.74151
iter: 7, loss_in_train: 22.27635, loss_in_test: 30.74151
iter: 8, loss_in_train: 22.27635, loss_in_test: 30.74151
iter: 9, loss_in_train: 22.27635, loss_in_test: 30.74151
iter: 10, loss_in_train: 22.27635, loss_in_test: 30.74151
iter: 11, loss_in_train: 22.27635, loss_in_test: 30.74151
iter: 12, loss_in_train: 22.27635, loss_in_test: 30.74151
iter: 13, loss_in_train: 22.27635, loss_in_test: 30.74151
iter: 14, loss_in_train: 22.27635, loss_in_test: 30.74152
iter: 15, loss_in_train: 22.27635, loss_in_test: 30.74152
iter: 16, loss_in_train: 22.27635, loss_in_test: 30.74152
iter: 17, loss_in_train: 22.27635, loss_in_test: 30.74152
iter: 18, loss_in_train